# 1. Про датасет и мертики
Выбранный датасет https://www.kaggle.com/datasets/sadhliroomyprime/cricket-semantic-segmentation/code
Датасет содержит изображения с крикетных матчей, где каждый объект (игроки, поле, разметка, снаряжение) размечен по классам. С помощью модели можно отслеживать действий во время матчей, по результат строить новую тактику\
1.б Метрики:
- IoU (Jaccard Index)	Показывает, насколько хорошо пересекаются предсказанные и истинные пиксели объекта.
- Dice Score	Аналогично IoU, но с другим фокусом: более чувствителен к малым объектам
- Pixel Accuracy	Процент правильно классифицированных пикселей

dssdsdsdasda

In [ ]:
!pip install kaggle
!pip install segmentation-models-pytorch albumentations --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.7 MB/s eta 0:00:00


загружаем датасет с kaggle

In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d sadhliroomyprime/cricket-semantic-segmentation
!unzip -q cricket-semantic-segmentation.zip -d cricket_dataset

Dataset URL: https://www.kaggle.com/datasets/sadhliroomyprime/cricket-semantic-segmentation
License(s): Attribution 4.0 International (CC BY 4.0)


In [ ]:
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from segmentation_models_pytorch.utils.train import TrainEpoch, ValidEpoch
import numpy as np
from sklearn.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import os, json, cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from glob import glob
import albumentations as A
from albumentations.pytorch import ToTensorV2

class CustomSegmentationDataset(Dataset):

    def __init__(self, dataset_root, transformations=None, image_extensions=[".jpg", ".png", ".jpeg"]):
        self.transformations = transformations
        self.root = dataset_root
        self.image_extensions = image_extensions
        self.image_paths = self.get_image_paths(is_ground_truth=False)
        self.mask_paths = self.get_image_paths(is_ground_truth=True)
        self.num_classes = len(self.read_json(os.path.join(dataset_root, "classes/classes.json")))

    def __len__(self):
        return len(self.image_paths)

    def read_json(self, path):
        return json.load(open(path))

    def get_all_image_files(self):
        return sorted(glob(f"{self.root}/images/*"))

    def get_image_id(self, path):
        return os.path.splitext(os.path.basename(path))[0]

    def get_image_paths(self, is_ground_truth=False):
        files = self.get_all_image_files()
        if is_ground_truth:
            return [path for path in files if "fuse" in self.get_image_id(path)]
        else:
            return [path for path in files if "___" not in self.get_image_id(path)]

    def read_image(self, path):
        return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

    def apply_transformations(self, image, mask):
        transformed = self.transformations(image=image, mask=mask)
        return transformed["image"], transformed["mask"]

    def __getitem__(self, index):
        image = self.read_image(self.image_paths[index])
        mask = self.read_image(self.mask_paths[index])
        if self.transformations:
            image, mask = self.apply_transformations(image, mask)
        return image, torch.div(mask, self.num_classes * 4).long()

In [ ]:
batch_size = 8
learning_rate = 1e-3

In [ ]:
transformations = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(transpose_mask=True)
])

dataset = CustomSegmentationDataset(
    dataset_root="/content/cricket_dataset/www.acmeai.tech ODataset 4 - Cricket Semantic Segmentation",
    transformations=transformations
)
num_classes = dataset.num_classes

train_len = int(len(dataset) * 0.9)
val_len = int(len(dataset) * 0.05)
test_len = len(dataset) - (train_len + val_len)

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

print(f"\nTraining images: {len(train_set)}")
print(f"Validation images: {len(val_set)}")
print(f"Testing images: {len(test_set)}\n")

train_loader = DataLoader(train_set, batch_size=batch_size*2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
num_classes


Training images: 268
Validation images: 14
Testing images: 16



9

In [ ]:
import segmentation_models_pytorch as smp
model = smp.Unet(
    encoder_name="resnet34",
    classes=num_classes,
    encoder_weights="imagenet",
    activation=None,
    decoder_channels=[256, 128, 64, 32, 16]
)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def flatten_tensor(tensor):
    return tensor.contiguous().view(-1)

def pixel_accuracy(preds, targets):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    assert preds.shape == targets.shape
    correct = (preds == targets).sum().item()
    total = targets.numel()
    return correct / total

def mean_iou(preds, targets, num_classes, eps=1e-10):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    preds = flatten_tensor(preds)
    targets = flatten_tensor(targets)

    ious = []
    for cls in range(num_classes):
        pred_mask = preds == cls
        target_mask = targets == cls
        intersection = torch.logical_and(pred_mask, target_mask).sum().float()
        union = torch.logical_or(pred_mask, target_mask).sum().float()
        if union == 0:
            continue
        iou = (intersection + eps) / (union + eps)
        ious.append(iou.item())
    return np.mean(ious) if ious else 0.0

def dice_score(preds, targets, num_classes, eps=1e-6):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)

    dice = 0.0
    for cls in range(num_classes):
        pred_mask = (preds == cls).float()
        target_mask = (targets == cls).float()
        intersection = (pred_mask * target_mask).sum()
        union = pred_mask.sum() + target_mask.sum()
        dice += (2.0 * intersection + eps) / (union + eps)
    return dice / num_classes


In [ ]:
def train_model(epochs):
    model.to(device)
    print("Starting training...")

    for epoch in range(1, epochs + 1):
        model.train()
        total_train_loss, total_train_iou, total_train_acc, train_dice_score = 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            predictions = model(images)
            loss = loss_function(predictions, torch.argmax(masks, dim=1))

            total_train_loss += loss.item()
            total_train_iou += mean_iou(predictions, masks, num_classes)
            total_train_acc += pixel_accuracy(predictions, masks)
            train_dice_score += dice_score(predictions, masks, num_classes)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        total_val_loss, total_val_iou, total_val_acc, val_dice_score = 0, 0, 0, 0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)

                predictions = model(images)

                total_val_loss += loss_function(predictions, torch.argmax(masks, dim=1)).item()
                total_val_iou += mean_iou(predictions, masks, num_classes)
                total_val_acc += pixel_accuracy(predictions, masks)
                val_dice_score += dice_score(predictions, masks, num_classes)


        print("\n===========================")
        print(f"Epoch {epoch} Summary:")
        print(f"Train Loss     : {total_train_loss / len(train_loader):.3f}")
        print(f"Train Accuracy : {total_train_acc / len(train_loader):.3f}")
        print(f"Train IoU     : {total_train_iou / len(train_loader):.3f}")
        print(f"Train Dice     : {train_dice_score / len(train_loader):.3f}")
        print(f"Val Loss       : {total_val_loss / len(val_loader):.3f}")
        print(f"Val Accuracy   : {total_val_acc / len(val_loader):.3f}")
        print(f"Val IoU       : {total_val_iou / len(val_loader):.3f}")
        print(f"Val Dice       : {val_dice_score / len(val_loader):.3f}")
        print("===========================\n")


    print("Training completed.")

train_model(epochs=10)

Starting training...

Epoch 1 Summary:
Train Loss     : 1.415
Train Accuracy : 0.634
Train IoU     : 0.081
Train Dice     : 0.138
Val Loss       : 0.954
Val Accuracy   : 0.941
Val IoU       : 0.109
Val Dice       : 0.116


Epoch 2 Summary:
Train Loss     : 0.414
Train Accuracy : 0.980
Train IoU     : 0.266
Train Dice     : 0.672
Val Loss       : 0.251
Val Accuracy   : 0.985
Val IoU       : 0.328
Val Dice       : 0.777


Epoch 3 Summary:
Train Loss     : 0.163
Train Accuracy : 0.981
Train IoU     : 0.327
Train Dice     : 0.777
Val Loss       : 0.121
Val Accuracy   : 0.985
Val IoU       : 0.328
Val Dice       : 0.777


Epoch 4 Summary:
Train Loss     : 0.101
Train Accuracy : 0.980
Train IoU     : 0.329
Train Dice     : 0.778
Val Loss       : 0.086
Val Accuracy   : 0.985
Val IoU       : 0.332
Val Dice       : 0.779


Epoch 5 Summary:
Train Loss     : 0.077
Train Accuracy : 0.981
Train IoU     : 0.381
Train Dice     : 0.806
Val Loss       : 0.066
Val Accuracy   : 0.987
Val IoU       : 0.38

2.b Выводы
Значения IoU и Dice стабильно росли\
Pixel Accuracy оставалась высокой и стабильной (выше 0.979) на всём протяжении обучения. Правда, точность большая из-за фона.\
Loss уменьшалась как на трейне, так и на валидации, что говорит о хорошем обучении модели без переобучения.

In [ ]:

def tensor_to_image(t):
    undo_transforms = tfs.Compose([
        tfs.Normalize(mean=[0., 0., 0.], std=[1/0.229, 1/0.224, 1/0.225]),
        tfs.Normalize(mean=[-0.485, -0.456, -0.406], std=[1., 1., 1.])
    ])

    is_rgb = len(t.shape) == 3 and t.shape[0] == 3

    if is_rgb:
        return (undo_transforms(t) * 255).detach().cpu().permute(1, 2, 0).numpy().astype(np.uint8)
    else:
        return (t * 255).detach().cpu().numpy().astype(np.uint8)

def evaluate_model_and_display(model, test_loader, device, num_classes, num_images=16):
    model.eval()
    model.to(device)

    total_test_acc, total_test_iou, total_test_dice = 0, 0, 0
    ims, gts, preds = [], [], []

    with torch.no_grad():
        for idx, (images, masks) in enumerate(test_loader):
            images, masks = images.to(device), masks.to(device)
            predictions = model(images)

            total_test_acc += pixel_accuracy(predictions, masks)
            total_test_iou += mean_iou(predictions, masks, num_classes)
            total_test_dice += dice_score(predictions, masks, num_classes)

            preds_batch = torch.argmax(predictions, dim=1)

            if len(ims) < num_images:
                ims.append(images.cpu())
                preds.append(preds_batch.cpu())
                gts.append(masks.cpu())

            if len(ims) >= num_images:
                break

    print("\n========= TEST RESULTS =========")
    print(f"Test Accuracy : {total_test_acc / num_images:.4f}")
    print(f"Test IoU      : {total_test_iou / num_images:.4f}")
    print(f"Test Dice     : {total_test_dice / num_images:.4f}")
    print("================================\n")

    cols = 2
    rows = num_images

    plt.figure(figsize=(20, 6 * rows))
    count = 1
    for idx, (im, pred) in enumerate(zip(ims, preds)):
        if idx == num_images:
            break

        plt.subplot(rows, cols, count)
        plt.imshow(tensor_to_image(im.squeeze(0)))
        plt.axis('off')
        plt.title("Original Image")
        count += 1

        plt.subplot(rows, cols, count)
        plt.imshow(pred.squeeze(0), cmap='tab20')
        plt.title("Predicted Mask")
        plt.axis('off')
        count += 1

    plt.tight_layout()
    plt.show()

In [ ]:
evaluate_model_and_display(model, test_loader, device, num_classes)

Output hidden; open in https://colab.research.google.com to view.

Базовая модель (Unet с энкодером resnet34 и стандартными аугментациями) показала высокую точность 98.5%, что указывает на уверенное различение фона и объектов. Однако ключевые метрики сегментации — IoU (0.6322) и Dice (0.9088) — демонстрируют, что несмотря на хорошую общую классификацию пикселей, точность сегментации по границам и перекрытию масок всё ещё имеет потенциал для улучшения, особенно в условиях сложных сцен или мелких объектов.

## 3. Улучшение бейзлайна

3.a Предполежнные гипотезы \
можно поменять encoder на resnet50 и попробовать сочетание  CrossEntropyLoss и IoU Loss (гибридная кросс-энтропия) + можно попробовать улучшить аугментации

3.bcde

In [ ]:
batch_size = 8
learning_rate = 5e-4

In [ ]:
transformations = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2(transpose_mask=True)
])


dataset = CustomSegmentationDataset(
    dataset_root="/content/cricket_dataset/www.acmeai.tech ODataset 4 - Cricket Semantic Segmentation",
    transformations=transformations
)
num_classes = dataset.num_classes

train_len = int(len(dataset) * 0.9)
val_len = int(len(dataset) * 0.05)
test_len = len(dataset) - (train_len + val_len)

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

print(f"\nTraining images: {len(train_set)}")
print(f"Validation images: {len(val_set)}")
print(f"Testing images: {len(test_set)}\n")

train_loader = DataLoader(train_set, batch_size=batch_size*2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
num_classes


Training images: 268
Validation images: 14
Testing images: 16



9

In [ ]:
import segmentation_models_pytorch as smp
model = smp.Unet(
    encoder_name="resnet50",
    classes=num_classes,
    encoder_weights="imagenet",
    decoder_channels=[256, 128, 64, 32, 16]
)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def flatten_tensor(tensor):
    return tensor.contiguous().view(-1)

def pixel_accuracy(preds, targets):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    assert preds.shape == targets.shape
    correct = (preds == targets).sum().item()
    total = targets.numel()
    return correct / total

def mean_iou(preds, targets, num_classes, eps=1e-10):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    preds = flatten_tensor(preds)
    targets = flatten_tensor(targets)

    ious = []
    for cls in range(num_classes):
        pred_mask = preds == cls
        target_mask = targets == cls
        intersection = torch.logical_and(pred_mask, target_mask).sum().float()
        union = torch.logical_or(pred_mask, target_mask).sum().float()
        if union == 0:
            continue
        iou = (intersection + eps) / (union + eps)
        ious.append(iou.item())
    return np.mean(ious) if ious else 0.0

def dice_score(preds, targets, num_classes, eps=1e-6):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)

    dice = 0.0
    for cls in range(num_classes):
        pred_mask = (preds == cls).float()
        target_mask = (targets == cls).float()
        intersection = (pred_mask * target_mask).sum()
        union = pred_mask.sum() + target_mask.sum()
        dice += (2.0 * intersection + eps) / (union + eps)
    return dice / num_classes

def iou_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)

    device = target.device
    one_hot_target = torch.eye(num_classes, device=device)[target.long()]

    if one_hot_target.dim() == 4:
        one_hot_target = one_hot_target.permute(0, 3, 1, 2)

    intersection = (pred * one_hot_target).sum(dim=(0, 2, 3))
    union = pred.sum(dim=(0, 2, 3)) + one_hot_target.sum(dim=(0, 2, 3)) - intersection

    iou = intersection / (union + smooth)
    return 1 - iou.mean()


def combined_ce_iou_loss(pred, target, alpha=0.45):
    ce = loss_function(pred, target)
    iou = iou_loss(pred, target)
    return alpha * ce + (1 - alpha) * iou

In [ ]:
def train_model(epochs):
    model.to(device)
    print("Starting training...")

    for epoch in range(1, epochs + 1):
        model.train()
        total_train_loss, total_train_iou, total_train_acc, train_dice_score = 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            predictions = model(images)
            loss = combined_ce_iou_loss(predictions, torch.argmax(masks, dim=1))

            total_train_loss += loss.item()
            total_train_iou += mean_iou(predictions, masks, num_classes)
            total_train_acc += pixel_accuracy(predictions, masks)
            train_dice_score += dice_score(predictions, masks, num_classes)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        total_val_loss, total_val_iou, total_val_acc, val_dice_score = 0, 0, 0, 0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)

                predictions = model(images)

                total_val_loss += combined_ce_iou_loss(predictions, torch.argmax(masks, dim=1)).item()
                total_val_iou += mean_iou(predictions, masks, num_classes)
                total_val_acc += pixel_accuracy(predictions, masks)
                val_dice_score += dice_score(predictions, masks, num_classes)


        print("\n===========================")
        print(f"Epoch {epoch} Summary:")
        print(f"Train Loss     : {total_train_loss / len(train_loader):.3f}")
        print(f"Train Accuracy : {total_train_acc / len(train_loader):.3f}")
        print(f"Train IoU     : {total_train_iou / len(train_loader):.3f}")
        print(f"Train Dice     : {train_dice_score / len(train_loader):.3f}")
        print(f"Val Loss       : {total_val_loss / len(val_loader):.3f}")
        print(f"Val Accuracy   : {total_val_acc / len(val_loader):.3f}")
        print(f"Val IoU       : {total_val_iou / len(val_loader):.3f}")
        print(f"Val Dice       : {val_dice_score / len(val_loader):.3f}")
        print("===========================\n")


    print("Training completed.")

train_model(epochs=10)

Starting training...

Epoch 1 Summary:
Train Loss     : 1.407
Train Accuracy : 0.573
Train IoU     : 0.073
Train Dice     : 0.092
Val Loss       : 2.282
Val Accuracy   : 0.666
Val IoU       : 0.083
Val Dice       : 0.106


Epoch 2 Summary:
Train Loss     : 1.038
Train Accuracy : 0.937
Train IoU     : 0.124
Train Dice     : 0.142
Val Loss       : 0.947
Val Accuracy   : 0.949
Val IoU       : 0.135
Val Dice       : 0.158


Epoch 3 Summary:
Train Loss     : 0.814
Train Accuracy : 0.973
Train IoU     : 0.153
Train Dice     : 0.234
Val Loss       : 0.762
Val Accuracy   : 0.978
Val IoU       : 0.180
Val Dice       : 0.381


Epoch 4 Summary:
Train Loss     : 0.657
Train Accuracy : 0.979
Train IoU     : 0.273
Train Dice     : 0.540
Val Loss       : 0.615
Val Accuracy   : 0.983
Val IoU       : 0.378
Val Dice       : 0.722


Epoch 5 Summary:
Train Loss     : 0.573
Train Accuracy : 0.983
Train IoU     : 0.459
Train Dice     : 0.797
Val Loss       : 0.548
Val Accuracy   : 0.987
Val IoU       : 0.54

Модель на улучшенном бейзлайне показала более лучшую метрику IoU на валидации
так можно заметить переобучение

In [ ]:
evaluate_model_and_display(model, test_loader, device, num_classes)

Output hidden; open in https://colab.research.google.com to view.

3.e После улучшений точность достигла 99.33%, средний показатель IoU составил 0.7087, а Dice-коэффициент вырос до 0.9362. Эти результаты превосходят исходный бейзлайн

# 4. Собственная реализация

4.a имплементации собственной модели

In [175]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class CustomUNet(nn.Module):
    def __init__(self, out_channels=1, features=[64, 128, 256, 512, 1024]):
        super().__init__()
        self.encoder = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        in_channels=3
        for feat in features:
            self.encoder.append(ConvBlock(in_channels, feat))
            in_channels = feat

        self.bottleneck = nn.Sequential(
            ConvBlock(features[-1], features[-1]*2),
            ConvBlock(features[-1]*2, features[-1]*2)
        )
        self.upconvs = nn.ModuleList()
        self.decoder = nn.ModuleList()
        for feat in reversed(features):
            self.upconvs.append(nn.ConvTranspose2d(feat*2, feat, kernel_size=2, stride=2))
            self.decoder.append(ConvBlock(feat*2, feat))

        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []
        for down in self.encoder:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(len(self.upconvs)):
            x = self.upconvs[idx](x)
            skip = skip_connections[idx]
            if x.shape != skip.shape:
                x = F.interpolate(x, size=skip.shape[2:])
            x = torch.cat((skip, x), dim=1)
            x = self.decoder[idx](x)

        return self.final_conv(x)

In [176]:
batch_size = 8
learning_rate = 1e-3

In [177]:
transformations = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2(transpose_mask=True)
])


dataset = CustomSegmentationDataset(
    dataset_root="/content/cricket_dataset/www.acmeai.tech ODataset 4 - Cricket Semantic Segmentation",
    transformations=transformations
)
num_classes = dataset.num_classes

train_len = int(len(dataset) * 0.9)
val_len = int(len(dataset) * 0.05)
test_len = len(dataset) - (train_len + val_len)

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

print(f"\nTraining images: {len(train_set)}")
print(f"Validation images: {len(val_set)}")
print(f"Testing images: {len(test_set)}\n")

train_loader = DataLoader(train_set, batch_size=batch_size*2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
num_classes


Training images: 268
Validation images: 14
Testing images: 16



9

In [182]:
model = CustomUNet(num_classes)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

4.b ниже все аналогично, что уже было

In [183]:
def flatten_tensor(tensor):
    return tensor.contiguous().view(-1)

def pixel_accuracy(preds, targets):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    assert preds.shape == targets.shape
    correct = (preds == targets).sum().item()
    total = targets.numel()
    return correct / total

def mean_iou(preds, targets, num_classes, eps=1e-10):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    preds = flatten_tensor(preds)
    targets = flatten_tensor(targets)

    ious = []
    for cls in range(num_classes):
        pred_mask = preds == cls
        target_mask = targets == cls
        intersection = torch.logical_and(pred_mask, target_mask).sum().float()
        union = torch.logical_or(pred_mask, target_mask).sum().float()
        if union == 0:
            continue
        iou = (intersection + eps) / (union + eps)
        ious.append(iou.item())
    return np.mean(ious) if ious else 0.0

def dice_score(preds, targets, num_classes, eps=1e-6):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)

    dice = 0.0
    for cls in range(num_classes):
        pred_mask = (preds == cls).float()
        target_mask = (targets == cls).float()
        intersection = (pred_mask * target_mask).sum()
        union = pred_mask.sum() + target_mask.sum()
        dice += (2.0 * intersection + eps) / (union + eps)
    return dice / num_classes


In [185]:
def train_model(epochs):
    model.to(device)
    print("Starting training...")

    for epoch in range(1, epochs + 1):
        model.train()
        total_train_loss, total_train_iou, total_train_acc, train_dice_score = 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            predictions = model(images)
            loss = loss_function(predictions, torch.argmax(masks, dim=1))

            total_train_loss += loss.item()
            total_train_iou += mean_iou(predictions, masks, num_classes)
            total_train_acc += pixel_accuracy(predictions, masks)
            train_dice_score += dice_score(predictions, masks, num_classes)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        total_val_loss, total_val_iou, total_val_acc, val_dice_score = 0, 0, 0, 0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)

                predictions = model(images)

                total_val_loss += loss_function(predictions, torch.argmax(masks, dim=1)).item()
                total_val_iou += mean_iou(predictions, masks, num_classes)
                total_val_acc += pixel_accuracy(predictions, masks)
                val_dice_score += dice_score(predictions, masks, num_classes)


        print("\n===========================")
        print(f"Epoch {epoch} Summary:")
        print(f"Train Loss     : {total_train_loss / len(train_loader):.3f}")
        print(f"Train Accuracy : {total_train_acc / len(train_loader):.3f}")
        print(f"Train IoU     : {total_train_iou / len(train_loader):.3f}")
        print(f"Train Dice     : {train_dice_score / len(train_loader):.3f}")
        print(f"Val Loss       : {total_val_loss / len(val_loader):.3f}")
        print(f"Val Accuracy   : {total_val_acc / len(val_loader):.3f}")
        print(f"Val IoU       : {total_val_iou / len(val_loader):.3f}")
        print(f"Val Dice       : {val_dice_score / len(val_loader):.3f}")
        print("===========================\n")


    print("Training completed.")

train_model(epochs=20)

Starting training...

Epoch 1 Summary:
Train Loss     : 1.530
Train Accuracy : 0.776
Train IoU     : 0.088
Train Dice     : 0.097
Val Loss       : 4.217
Val Accuracy   : 0.410
Val IoU       : 0.052
Val Dice       : 0.177


Epoch 2 Summary:
Train Loss     : 0.812
Train Accuracy : 0.980
Train IoU     : 0.244
Train Dice     : 0.587
Val Loss       : 0.654
Val Accuracy   : 0.982
Val IoU       : 0.327
Val Dice       : 0.777


Epoch 3 Summary:
Train Loss     : 0.471
Train Accuracy : 0.981
Train IoU     : 0.327
Train Dice     : 0.777
Val Loss       : 0.430
Val Accuracy   : 0.982
Val IoU       : 0.245
Val Dice       : 0.666


Epoch 4 Summary:
Train Loss     : 0.290
Train Accuracy : 0.981
Train IoU     : 0.322
Train Dice     : 0.770
Val Loss       : 0.268
Val Accuracy   : 0.982
Val IoU       : 0.327
Val Dice       : 0.777


Epoch 5 Summary:
Train Loss     : 0.201
Train Accuracy : 0.981
Train IoU     : 0.327
Train Dice     : 0.777
Val Loss       : 0.182
Val Accuracy   : 0.982
Val IoU       : 0.32

In [186]:
evaluate_model_and_display(model, test_loader, device, num_classes)

Output hidden; open in https://colab.research.google.com to view.

4.c Качество собственной реализации весьма низкое, чтобы хоть как-то улучшить результат пришлось увеличить количество эпох \
4.d собственная реализация показала существенно худшие результаты, чем Unet, так как в Unet предобученные энкодеры \
4.e модель показала не такой плохой результат, но все же хуже чем у Unet'а

4.fg аналогично предыдущему

In [187]:
batch_size = 8
learning_rate = 5e-4

In [188]:
transformations = A.Compose([
    A.Resize(256, 256),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
    ToTensorV2(transpose_mask=True)
])


dataset = CustomSegmentationDataset(
    dataset_root="/content/cricket_dataset/www.acmeai.tech ODataset 4 - Cricket Semantic Segmentation",
    transformations=transformations
)
num_classes = dataset.num_classes

train_len = int(len(dataset) * 0.9)
val_len = int(len(dataset) * 0.05)
test_len = len(dataset) - (train_len + val_len)

train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

print(f"\nTraining images: {len(train_set)}")
print(f"Validation images: {len(val_set)}")
print(f"Testing images: {len(test_set)}\n")

train_loader = DataLoader(train_set, batch_size=batch_size*2, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)
num_classes


Training images: 268
Validation images: 14
Testing images: 16



9

In [189]:
model = CustomUNet(num_classes)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [190]:
def flatten_tensor(tensor):
    return tensor.contiguous().view(-1)

def pixel_accuracy(preds, targets):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    assert preds.shape == targets.shape
    correct = (preds == targets).sum().item()
    total = targets.numel()
    return correct / total

def mean_iou(preds, targets, num_classes, eps=1e-10):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)
    preds = flatten_tensor(preds)
    targets = flatten_tensor(targets)

    ious = []
    for cls in range(num_classes):
        pred_mask = preds == cls
        target_mask = targets == cls
        intersection = torch.logical_and(pred_mask, target_mask).sum().float()
        union = torch.logical_or(pred_mask, target_mask).sum().float()
        if union == 0:
            continue
        iou = (intersection + eps) / (union + eps)
        ious.append(iou.item())
    return np.mean(ious) if ious else 0.0

def dice_score(preds, targets, num_classes, eps=1e-6):
    preds = torch.argmax(torch.nn.functional.softmax(preds, dim=1), dim=1)
    if targets.ndimension() == 4 and targets.size(1) > 1:
        targets = torch.argmax(targets, dim=1)

    dice = 0.0
    for cls in range(num_classes):
        pred_mask = (preds == cls).float()
        target_mask = (targets == cls).float()
        intersection = (pred_mask * target_mask).sum()
        union = pred_mask.sum() + target_mask.sum()
        dice += (2.0 * intersection + eps) / (union + eps)
    return dice / num_classes

def iou_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)

    device = target.device
    one_hot_target = torch.eye(num_classes, device=device)[target.long()]

    if one_hot_target.dim() == 4:
        one_hot_target = one_hot_target.permute(0, 3, 1, 2)

    intersection = (pred * one_hot_target).sum(dim=(0, 2, 3))
    union = pred.sum(dim=(0, 2, 3)) + one_hot_target.sum(dim=(0, 2, 3)) - intersection

    iou = intersection / (union + smooth)
    return 1 - iou.mean()


def combined_ce_iou_loss(pred, target, alpha=0.45):
    ce = loss_function(pred, target)
    iou = iou_loss(pred, target)
    return alpha * ce + (1 - alpha) * iou

In [192]:
def train_model(epochs):
    model.to(device)
    print("Starting training...")

    for epoch in range(1, epochs + 1):
        model.train()
        total_train_loss, total_train_iou, total_train_acc, train_dice_score = 0, 0, 0, 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            predictions = model(images)
            loss = combined_ce_iou_loss(predictions, torch.argmax(masks, dim=1))

            total_train_loss += loss.item()
            total_train_iou += mean_iou(predictions, masks, num_classes)
            total_train_acc += pixel_accuracy(predictions, masks)
            train_dice_score += dice_score(predictions, masks, num_classes)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        model.eval()
        total_val_loss, total_val_iou, total_val_acc, val_dice_score = 0, 0, 0, 0
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)

                predictions = model(images)

                total_val_loss += combined_ce_iou_loss(predictions, torch.argmax(masks, dim=1)).item()
                total_val_iou += mean_iou(predictions, masks, num_classes)
                total_val_acc += pixel_accuracy(predictions, masks)
                val_dice_score += dice_score(predictions, masks, num_classes)


        print("\n===========================")
        print(f"Epoch {epoch} Summary:")
        print(f"Train Loss     : {total_train_loss / len(train_loader):.3f}")
        print(f"Train Accuracy : {total_train_acc / len(train_loader):.3f}")
        print(f"Train IoU     : {total_train_iou / len(train_loader):.3f}")
        print(f"Train Dice     : {train_dice_score / len(train_loader):.3f}")
        print(f"Val Loss       : {total_val_loss / len(val_loader):.3f}")
        print(f"Val Accuracy   : {total_val_acc / len(val_loader):.3f}")
        print(f"Val IoU       : {total_val_iou / len(val_loader):.3f}")
        print(f"Val Dice       : {val_dice_score / len(val_loader):.3f}")
        print("===========================\n")


    print("Training completed.")

train_model(epochs=20)

Starting training...

Epoch 1 Summary:
Train Loss     : 1.239
Train Accuracy : 0.799
Train IoU     : 0.096
Train Dice     : 0.144
Val Loss       : 6.777
Val Accuracy   : 0.059
Val IoU       : 0.009
Val Dice       : 0.180


Epoch 2 Summary:
Train Loss     : 1.017
Train Accuracy : 0.975
Train IoU     : 0.132
Train Dice     : 0.286
Val Loss       : 0.999
Val Accuracy   : 0.932
Val IoU       : 0.110
Val Dice       : 0.163


Epoch 3 Summary:
Train Loss     : 0.896
Train Accuracy : 0.980
Train IoU     : 0.155
Train Dice     : 0.398
Val Loss       : 1.019
Val Accuracy   : 0.935
Val IoU       : 0.117
Val Dice       : 0.219


Epoch 4 Summary:
Train Loss     : 0.805
Train Accuracy : 0.981
Train IoU     : 0.175
Train Dice     : 0.450
Val Loss       : 0.850
Val Accuracy   : 0.971
Val IoU       : 0.230
Val Dice       : 0.554


Epoch 5 Summary:
Train Loss     : 0.734
Train Accuracy : 0.981
Train IoU     : 0.228
Train Dice     : 0.607
Val Loss       : 0.757
Val Accuracy   : 0.970
Val IoU       : 0.19

In [193]:
evaluate_model_and_display(model, test_loader, device, num_classes)

Output hidden; open in https://colab.research.google.com to view.

4.h Модель показала результат похуже на улучшенном бейзлайне\
4.i Модель показала существенно хуже результат, но это и понятно, так как не используется предобученные эндкодеры\
4.j Собственная реализация модели продемонстрировала значительно более низкие метрики по сравнению с UNet. Это ожидаемый результат, так как модель обучается "с нуля" и не использует предобученные энкодеры. Модель способносна распознавать крупные объекты, но недостаточна способносна к точной сегментации. Основное отставание наблюдается по метрикам IoU и Dice, что подчёркивает необходимость использования более сложных архитектур или предобученных компонентов для достижения конкурентного качества.

